In [1]:
# import sys
# import gymnasium as gym
# sys.modules["gym"] = gym
import gym
gym.__version__

'0.27.1'

In [3]:
import gnwrapper

In [4]:
import os
import numpy as np
import torch
print(torch.cuda.is_available())

True


In [5]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

In [6]:
env = gym.make("CartPole-v1")

In [ ]:
expert = PPO(
    policy=MlpPolicy,
    env=env,
    seed=0,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
    n_steps=64,
)
expert.learn(1_000_000, progress_bar=True)
expert.save("ppo_CartPole_expert")

  68% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 678,529/1,000,000  [ 0:06:25 < 0:03:03 , 1,761 it/s ]
  68% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 678,529/1,000,000  [ 0:06:25 < 0:03:03 , 1,761 it/s ]

In [9]:
expert = PPO.load("ppo_CartPole_expert.zip", print_system_info=True)

== CURRENT SYSTEM INFO ==
- OS: Linux-5.15.0-58-generic-x86_64-with-glibc2.35 # 64-Ubuntu SMP Thu Jan 5 11:43:13 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 2.0.0a0
- PyTorch: 1.13.1+cu117
- GPU Enabled: True
- Numpy: 1.24.1
- Gym: 0.27.1

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.0-58-generic-x86_64-with-glibc2.35 # 64-Ubuntu SMP Thu Jan 5 11:43:13 UTC 2023
- Python: 3.10.6
- Stable-Baselines3: 2.0.0a0
- PyTorch: 1.13.1+cu117
- GPU Enabled: True
- Numpy: 1.24.1
- Gym: 0.27.1



In [10]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(expert, env, 10)
print(reward)

/home/vlad1slav/Diploma/rl-venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


500.0


In [11]:
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np

rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    DummyVecEnv([lambda: RolloutInfoWrapper(env)]),
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)
transitions = rollout.flatten_trajectories(rollouts)

ValueError: too many values to unpack (expected 4)

In [ ]:
print(
    f"""The `rollout` function generated a list of {len(rollouts)} {type(rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)

In [ ]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward before training: {reward_before_training}")

In [ ]:
bc_trainer.train(n_epochs=1)
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward after training: {reward_after_training}")
